# Time Series Analysis and Forecasting

This is a series of notebooks to support lectures on _Time series analysis and forecast_.

![](https://user-images.githubusercontent.com/49638680/154160268-cf39a1ec-3557-4940-8853-d06fc7a79acf.png)

## Time series Analysis

We start by exploring pandas method to treat time series. 
In paticular, over this notebook we are going to analyse a dataframe and how to deal with pandas time index.

### Import data

Let's have a look at the structure of a real-life time series. We will use both known and new Pandas tools, as well as Matplotlib and Seaborn.
Once we are able to analyse a time series, we will use the information we have inferred to predict its future.
In the meantime, here and there, we will think about the relationship with supervised learning.

![img](https://raw.githubusercontent.com/fbagattini/Lezioni/master/img/OPSD.png)

[OPSD database](https://open-power-system-data.org/) is a dataset provided by the famous open source project OPSD that provides data on installed generation capacity by country/technology, individual power plants (conventional and renewable), and time series data. The latter includes electricity consumption, spot prices, and wind and solar generation, both measured and derived from weather models.

The aim of this very cool project is to make life easier for power system modelers. The goal is to help researchers focus on research and avoid redundant work when collecting, preparing, and aggregating data.

In particular, in this lecture we are going to import the OPSD_Germany_consumption dataset. This reports Germany's electric energy consumption (in GWh) from 2006 to 2017. These data are part of the OPSD project.

Let's start by loading the dataset as a DataFrame. The only mandatory parameter of the read_csv function is the path of the .csv file we are loading. The head and tail functions respectively show the first and last records (5 by default) of the dataframe.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

import statsmodels.api as sm

%matplotlib inline
%config InlineBackend.figure_format = "retina"
plt.rcParams['figure.figsize'] = (15.0, 10.0)

In [2]:
data_url = 'https://raw.githubusercontent.com/fbagattini/Lezioni/master/data/OPSD_Germany_consumption.csv'
df = pd.read_csv(data_url)
df.head()

,Date,Consumption
0,2006-01-01,1069.184
1,2006-01-02,1380.521
2,2006-01-03,1442.533
3,2006-01-04,1457.217
4,2006-01-05,1477.131


In [3]:
df.shape

(4383, 2)

Note how the `DataFrame` object in pandas _inherits_ the attribute `shape` from numpy. 
The dataframe contains $4383$ records: each row reports, in addition to the consumption, the date at which the record is observed.

Pandas, at loading time, provides the dataframe with an indexing structure. 
Via the synthax

```python
df.loc[position]
```

it is possible to access the element whose index is position.

In [4]:
df.loc[7]

Date           2006-01-08
Consumption      1207.985
Name: 7, dtype: object

Whe are used to associate the columns of a dataset to the features. What about `Date`? Is that a special feature? Which role could it have within supervised learning?

Let's have a look at its content.

In [5]:
df['Date'].dtype

dtype('O')

In Pandas, type 'O' stands for Object: a generic type, for our purposes we can consider it more broadly as a string.

In [6]:
df['Date'].loc[0]

'2006-01-01'

We want to give these data a structure. Meaning we want to be able to define an operation like

```python
df['Date'].loc[0] + year
```

having '2007-01-01' as result.

Hence, we aim at providing these records with a temporal structure.

### Time indexing

We are going to explore a pandas method to perform such conversion `to_datetime`. This will result in a new type of object: `Timestamp`.

In [7]:
pd.to_datetime('10-02-09')

Timestamp('2009-10-02 00:00:00')

Note how we got not only the date, but also the time for free.

The function has attemped to autonomously infer the date format: October 2, 2009. Through optional arguments we can drive this transformation.

Can you tell the difference between the two following cells?

In [8]:
pd.to_datetime('10-02-09', yearfirst=True)

Timestamp('2010-02-09 00:00:00')

In [9]:
pd.to_datetime('10-02-09', dayfirst=True)

Timestamp('2009-02-10 00:00:00')

The `to_datetime` method can handle several formats.

In [10]:
pd.to_datetime('7th of June 1990')

Timestamp('1990-06-07 00:00:00')

In [11]:
pd.to_datetime('Feb 10 1990')

Timestamp('1990-02-10 00:00:00')

As with individual strings, we can convert lists of strings; in our instance, we can transform the whole `Date` column.

In [12]:
df.Date = pd.to_datetime(df.Date)

In [13]:
df

,Date,Consumption
0,2006-01-01,1069.18400
1,2006-01-02,1380.52100
2,2006-01-03,1442.53300
3,2006-01-04,1457.21700
4,2006-01-05,1477.13100
...,...,...
4378,2017-12-27,1263.94091
4379,2017-12-28,1299.86398
4380,2017-12-29,1295.08753
4381,2017-12-30,1215.44897


In [14]:
df.loc[7]

Date           2006-01-08 00:00:00
Consumption               1207.985
Name: 7, dtype: object

In [15]:
df.dtypes

Date           datetime64[ns]
Consumption           float64
dtype: object

It is noteworthy that python stores temporal objects as 64-bits integers, so that to handle nanoseconds (ns) precision.

**Recap**: we have transformed a column of the dataframe into a special one, equipped with temporal logic.

### DateTimeIndex

As you might have noticed, we have one example per day (this is called _frequency_). Furthermore, we have already talked about the definition of a time series, _i.e._ a _realisation_ of a stochastic process thanks to its projection on a time interval.

$$\{x_1, \ldots , x_n \vert x_k = x(t_k) \} \, .$$

Hence, it is natural to use the time variable (the `Date` in this case) as an index. Pandas allows us to do so.

In [16]:
df.set_index('Date')

,Consumption
Date,
2006-01-01,1069.18400
2006-01-02,1380.52100
2006-01-03,1442.53300
2006-01-04,1457.21700
2006-01-05,1477.13100
...,...
2017-12-27,1263.94091
2017-12-28,1299.86398
2017-12-29,1295.08753


Great! let's call our dataset again.

In [17]:
df

,Date,Consumption
0,2006-01-01,1069.18400
1,2006-01-02,1380.52100
2,2006-01-03,1442.53300
3,2006-01-04,1457.21700
4,2006-01-05,1477.13100
...,...,...
4378,2017-12-27,1263.94091
4379,2017-12-28,1299.86398
4380,2017-12-29,1295.08753
4381,2017-12-30,1215.44897


Wait! 😲 

Nothing has happened to the dataframe. Why? Where did we go wrong?

Often, Pandas functions return a new dataframe, without modifying the original one. In such cases, we have to either overwrite it (option `inplace=True`) or create a new one (assigning a new variable).

In [18]:
df = df.set_index('Date')

df.head()

,Consumption
Date,
2006-01-01,1069.184
2006-01-02,1380.521
2006-01-03,1442.533
2006-01-04,1457.217
2006-01-05,1477.131


Now `df` is correctly indexed by `Date`. Let's have a look at the new index.

In [19]:
df.index

DatetimeIndex(['2006-01-01', '2006-01-02', '2006-01-03', '2006-01-04',
               '2006-01-05', '2006-01-06', '2006-01-07', '2006-01-08',
               '2006-01-09', '2006-01-10',
               ...
               '2017-12-22', '2017-12-23', '2017-12-24', '2017-12-25',
               '2017-12-26', '2017-12-27', '2017-12-28', '2017-12-29',
               '2017-12-30', '2017-12-31'],
              dtype='datetime64[ns]', name='Date', length=4383, freq=None)

It is not the usual `Index`, it is something more specific `DatetimeIndex`. Indeed, it has specific properties.

In [20]:
df.index.year

Int64Index([2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006,
            ...
            2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017],
           dtype='int64', name='Date', length=4383)

In [21]:
df.index.day

Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10,
            ...
            22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
           dtype='int64', name='Date', length=4383)

**Recap**: we have used the time information provided by our data (the date on which each record is observed) to index a dataframe.

### Indexing at loading time

Since the greatest force moving a developer is lazyness, we do not want to work on our dataframes in order to index them correctly. In other words we do not want to repeat the operations we have done above, _i.e._ converting to the right datetime format, creating a copy of our dataframe, changing the index to `Datetimeindex`, etc.

We can achieve the same goal in a compact way. Among the arguments of `read_csv`, there are:

* `parse_dates` 
* `index_col`

The former one accepts a boolean as value (hence`True` or `False`) setting whether to try parsing dates as `Datetime` objects, on the other hand the latter accepts a string or a list of strings indicating which columns should be used as index.

![](https://media.makeameme.org/created/ta-da.jpg)

In [22]:
df = pd.read_csv(data_url,
                 parse_dates=True,
                 index_col='Date')
df.head()

,Consumption
Date,
2006-01-01,1069.184
2006-01-02,1380.521
2006-01-03,1442.533
2006-01-04,1457.217
2006-01-05,1477.131


__Useful remark__: recall that it is possible to access the documentation of a function `func` by placing the cursor to the right of a function name and pressing
```
Shift Tab (Tab Tab)
```

Alternatively, we can run a cell like
```
func?
```

Let's now print some info about our dataframe.

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4383 entries, 2006-01-01 to 2017-12-31
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Consumption  4383 non-null   float64
dtypes: float64(1)
memory usage: 68.5 KB


**Recap**: with just one line of code we have loaded the OPSD_Germany_consumption dataset and set Date column as its time index.

### String and partial-string indexing

Now that records have a time reference attached to, we can easily access them through string indexing.

Let's suppose we want to know the electric consumption of Christmas 2015.

In [27]:
df.loc['2015-12-25']

Consumption    1047.277
Name: 2015-12-25 00:00:00, dtype: float64

And we want to compare this record with the average consumption of December 2015. It is quite easy to do so in pandas.

As with slicing in Python and NumPy, string indexing can be used with ':' to access time intervals.

In [29]:
df.loc['2015-12-1':'2015-12-31']

,Consumption
Date,
2015-12-01,1588.021
2015-12-02,1585.308
2015-12-03,1577.457
2015-12-04,1570.318
2015-12-05,1337.095
2015-12-06,1232.073
2015-12-07,1536.251
2015-12-08,1572.740
2015-12-09,1586.393


**Important note**: A noteworthy difference between pandas and plain python slicing is that in pandas both extrema of the interval are included in the result.

Another temporal indexing technique is partial-string indexing. By specifying the month and year, we extract only the records of December 2015.

In [30]:
df.loc['2015-12']

,Consumption
Date,
2015-12-01,1588.021
2015-12-02,1585.308
2015-12-03,1577.457
2015-12-04,1570.318
2015-12-05,1337.095
2015-12-06,1232.073
2015-12-07,1536.251
2015-12-08,1572.740
2015-12-09,1586.393


And here is the average consumption of December 2015.

In [31]:
df.loc['2015-12'].mean()

Consumption    1375.545516
dtype: float64

**Recap**: we have learnt how to access the elements of a time indexed dataframe.

### Exercise 1

Along with electric energy consumption, the [OPSD_Germany_all](https://raw.githubusercontent.com/fbagattini/Lezioni/master/data/OPSD_Germany_all.csv) dataset reports the daily production of solar and wind energy.

1. Load the dataset as a dataframe, using Date as time index. Solar energy production is not available until December 31, 2011; from the dataframe, select only the records following this date.

2. Create a column Renewable as the sum of solar and wind energy.

3. Compute the ratio between a) the total renewable production of September 2014 and b) the total electric consumption of the same month.

4. Using the indexing properties of Date (hint: `index.day_name()`), create the column 'Weekday' containing, for each record, the day of the week at which it's been observed (Monday, Tuesday,...).

5. Create the dataframe `df_sunday_wind` containing (only) Sunday wind production.

6. Compute the average Sunday wind production between January and March 2017 (included).